In [ ]:
%pylab inline
import openpmd_api as io
import numpy as np
from numba import cuda, jit
import math
from scipy.ndimage import gaussian_filter
import scipy.constants

## Things to do in the future

* specify a cutoff, behind all fields are written as 0 instead of a small value e.g. 1e-10 * field_max
* multi GPU / multi Node calculations  field size are limited by GPU memory size

## Things to note
* The Convolutional PML fields aren't changed

In [ ]:
# this cell is only required for testing the write proccess
try:
    del readSeries
    print("del readSeries")
except:
    pass
try:
    del writeSeries
    print("del writeSereies")
except:
    pass


## Set path to checkpoint

In [ ]:

# you can use one checkpoint explicitly checkpoint_0.h5
# or a time series with checkpoint_%T.h5
# the warning can be ignored when using the explicit checkpoint, we want just a single file
#path = "/bigdata/hplsim/scratch/spreng88/runs/bigFieldWithoutInit23/simOutput/checkpoints/checkpoint_0.h5"
readFile = "/bigdata/hplsim/scratch/spreng88/runs/old/restart/tmp_%T.h5"
outputFile = "/bigdata/hplsim/scratch/spreng88/runs/restart/write_%T.h5"

#readFile = outputFile
# io.Access.read_write only works with h5 files
readSeries = io.Series(readFile, io.Access.read_only)
writeSeries = io.Series(outputFile, io.Access.create)

## General setup

In [ ]:
nth = 1             # only use every nth - particle (for faster calculation for tests)

do_write = False       # True if the calculated field's should be written directly after calculations
                    # False if you want to look at the fields before writing them

# add field to iteration 0
iteration = readSeries.iterations[0] 

# Read attributes of the simulation that might be needed later
# Everything is calculated in PIConGPU units
# For SI units: length_si = value_pic * unit_length
cell_depth = iteration.get_attribute("cell_depth")          # z
cell_height = iteration.get_attribute("cell_height")        # y
cell_width = iteration.get_attribute("cell_width")          # x

unit_efield = iteration.get_attribute("unit_efield")
unit_bfield = iteration.get_attribute("unit_bfield")
unit_charge = iteration.get_attribute("unit_charge")
unit_mass = iteration.get_attribute("unit_mass")
unit_speed = iteration.get_attribute("unit_speed")
unit_length = iteration.get_attribute("unit_length")
unit_time = iteration.get_attribute("unit_time")

pi = scipy.constants.pi
c = scipy.constants.c / unit_speed
eps0 = iteration.get_attribute("eps0")
mue0 = iteration.get_attribute("mue0")

## Cuda functions

In [ ]:

@cuda.jit(device=True)
def particleV(px, py, pz, mass):
    """
    calculate particle speed from momentum
    """
    
    m2p2 = math.sqrt( (mass)**2 + px**2 + py**2 + pz**2)
    
    vx = px / m2p2 * c
    vy = py / m2p2 * c
    vz = pz / m2p2 * c
    
    return vx, vy, vz


@cuda.jit(device=True)
def param1(rx,ry,rz, rqx,rqy,rqz, vx,vy,vz):
    """
    calculate some parameters that could be reused
    r: position of field calculation
    rq: position of charge q at time t
    rq_tr: position of charge at retarded time tr
    """
    
    #
    # solution equation:
    # dt = t - tr   = time - retarded_time
    # r where we want the field
    # rq position of charge
    # |c*dt| = |r - (rq - v*dt)|
    # solve for dt:
    # => (c^2 - |v|^2) * dt^2 - 2(r*v - rq*v) * dt + 2 * r*rq - |r|^2 - |rq|^2
    #
    a = (c**2 - (vx**2 + vy**2 + vz**2))
    b = -2 * ((rx-rqx)*vx + (ry-rqy)*vy + (rz-rqz)*vz)
    d = 2 * (rx*rqx + ry*rqy + rz*rqz) - (rx**2 + ry**2 + rz**2) - (rqx**2 + rqy**2 + rqz**2)
    dt = (-b + math.sqrt(b**2 - 4*a*d)) / (2*a)
    
    rq_trx = rqx - vx * dt
    rq_try = rqy - vy * dt
    rq_trz = rqz - vz * dt

    dvx = rx - rq_trx
    dvy = ry - rq_try
    dvz = rz - rq_trz
    
    distance = math.sqrt(dvx**2 + dvy**2 + dvz**2)
    
    if distance == 0:
        distance = -1
    
    #n =  distanceVec / distance
    nx = dvx / distance
    ny = dvy / distance
    nz = dvz / distance
    
    return nx, ny, nz, distance, dvx, dvy, dvz #n, distance, distanceVec


@cuda.jit(device=True)
def retardedEFieldParallel(q, nx, ny, nz, distance, dvx, dvy, dvz, vx, vy, vz):
    """
    field for one position
    q: charge
    nx, ny, nz: unit vector to retarded position
    distance: distance to retarded position
    dvx, dvy, dvz: distance Vector to retarded position
    vx, vy, vz: velocity vector of the charge
    """
    
    if distance == -1:
        nx = 1
        ny = 0
        xz = 0
    
    factor = q / (4 * pi * eps0)
    
    ux = c * nx - vx
    uy = c * ny - vy
    uz = c * nz - vz
    

    scalar = factor * distance / (dvx*ux + dvy*uy + dvz*uz)**3 * (c**2 - (vx**2 + vy**2 + vz**2))
    
    return  scalar * ux, scalar * uy, scalar * uz


@cuda.jit(device=True)
def returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass):
    """calculate the fields for the position,
    currently ignoring the offset between the grid corner and field points
    because we will use smoothing in the end over the field anyway"""
    
    vx, vy, vz = particleV(px, py, pz, mass)
    
    ## E Field ------------------------------------------------------------
    nx, ny, nz, distancez, dvx, dvy, dvz = param1(rx, ry, rz, rqx, rqy, rqz, vx,vy,vz)
    ex, ey, ez = retardedEFieldParallel(q, nx, ny, nz, distancez, dvx, dvy, dvz , vx, vy, vz)

    
    ## B Field ------------------------------------------------------------
    bx = ( ny * ez - nz * ey ) / c
    by = ( nz * ex - nx * ez ) / c
    bz = ( nx * ey - ny * ex ) / c
    
    return ex, ey, ez, bx, by, bz

In [ ]:
@cuda.jit
def particleParallel(Ex,Ey,Ez, Bx,By,Bz, q_, rqx_,rqy_,rqz_, px_,py_,pz_, mass_, weighting_, xdim, ydim, zdim, particleCount):
    """
    Ex,Ey,Ez E field where new field is added
    Bx,By,Bz B -"-
    
    _ is on all input arrays read from checkpoint 
    q_ array from checkpoint with charges
    rq_ position data of particles
    p_ momentum of particles
    weighting_ macro particle weighting
    
    xdim, ydim, zdim length of dimension of Ex,Bx, ...
    Ex/Bx need to be 1D for atomic add
    """
    
    tix = cuda.threadIdx.x
    bix = cuda.blockIdx.x
    bdx = cuda.blockDim.x
    
    index = tix + bix * bdx    # particle index
    index = index
    
    if index < particleCount:
    
        q = q_[index] * weighting_[index]
        rqx = rqx_[index] * cell_width
        rqy = rqy_[index] * cell_height
        rqz = rqz_[index] * cell_depth
        px = px_[index]
        py = py_[index]
        pz = pz_[index]
        mass = mass_[index] * weighting_[index]

        for x in range(xdim):

            rx = x * cell_width
            rex = (x + 0.5) * cell_width

            for y in range(ydim):

                ry = y * cell_height
                rey = (y + 0.5) * cell_height

                for z in range(zdim):    
                    
                    rz = z * cell_depth
                    rez = (z + 0.5) * cell_depth

                    fieldIndex = x + y * xdim + z * xdim * ydim
                    
                    ex, ey, ez, bx, by, bz = returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass)
                    
                    cuda.atomic.add(Ex, fieldIndex, ex * nth)
                    cuda.atomic.add(Ey, fieldIndex, ey * nth)
                    cuda.atomic.add(Ez, fieldIndex, ez * nth)

                    cuda.atomic.add(Bx, fieldIndex, bx)
                    cuda.atomic.add(By, fieldIndex, by)
                    cuda.atomic.add(Bz, fieldIndex, bz)

In [ ]:
@cuda.jit
def FieldParallel(Ex,Ey,Ez, Bx,By,Bz, q_, rqx_,rqy_,rqz_, px_,py_,pz_, mass_, weighting_, xdim, ydim, zdim, particleCount):
    """
    Ex,Ey,Ez E field where new field is added
    Bx,By,Bz B -"-
    
    _ is on all input arrays read from checkpoint 
    q_ array from checkpoint with charges
    rq_ position data of particles
    p_ momentum of particles
    
    xdim, ydim, zdim length of dimension of Ex,Bx, ...
    Ex/Bx need to be 1D for atomic add
    """
    tix = cuda.threadIdx.x
    bix = cuda.blockIdx.x
    bdx = cuda.blockDim.x
    
    fieldIndex = tix + bix * bdx    # field index
    
    # calculate 3d coordinate from 1d field coordinate array
    # fieldIndex = x + y * xdim + z * xdim * ydim
    z = math.floor( fieldIndex / (xdim*ydim) )
    y = math.floor( (fieldIndex - z * xdim*ydim) / xdim )
    x = fieldIndex - z * xdim*ydim - y * xdim
    
    if fieldIndex < xdim*ydim*zdim:
        
        #for index in range(particleCount):
        # or add filter if you only want the field of some particles of that species
        for index in range(particleCount):
        
            q = q_[index] * weighting_[index]
            rqx = rqx_[index] * cell_width
            rqy = rqy_[index] * cell_height
            rqz = rqz_[index] * cell_depth
            px = px_[index]
            py = py_[index]
            pz = pz_[index]
            mass = mass_[index] * weighting_[index]
            
            rx = x * cell_width
            rex = (x + 0.5) * cell_width
            
            ry = y * cell_height
            rey = (y + 0.5) * cell_height
            
            rz = z * cell_depth
            rez = (z + 0.5) * cell_depth
            
            ex, ey, ez, bx, by, bz = returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass)
            if((x == 3)and(z%200==0)):
                pass
                #print(rqx,rqy,rqz)

            cuda.atomic.add(Ex, fieldIndex, ex * nth)
            cuda.atomic.add(Ey, fieldIndex, ey * nth)
            cuda.atomic.add(Ez, fieldIndex, ez * nth)

            cuda.atomic.add(Bx, fieldIndex, bx)
            cuda.atomic.add(By, fieldIndex, by)
            cuda.atomic.add(Bz, fieldIndex, bz)
            

In [ ]:
def copy_attributes(inputAtr, outputAtr):
    for attribute in inputAtr.attributes:
        #print("attribute copied:",attribute)
        attribute_value = inputAtr.get_attribute(attribute)
        outputAtr.set_attribute(attribute, attribute_value)
        
def copy_series_attributes(inputAtr, outputAtr):
    attributes_to_skip = ['openPMD', 'openPMDextension']
    for attribute in inputAtr.attributes:
        if attribute in attributes_to_skip:
            continue
        #print("attribute copied:",attribute)
        attribute_value = inputAtr.get_attribute(attribute)
        outputAtr.set_attribute(attribute, attribute_value)
        
def print_attributes(atr):
    try:
        for attribute in atr.attributes:
            print(attribute, atr.get_attribute(attribute))
        print()
    except:
        print('errro in print attributes')


def copy_particle_data(inputIteration, outputIteration):
    # copy the all particle data from input to output iteration

    global writeSeries
    
    for species in inputIteration.particles:
        copy_attributes(inputIteration.particles[species], outputIteration.particles[species])
        
        for species_data in inputIteration.particles[species]:
            
            copy_attributes(inputIteration.particles[species][species_data], outputIteration.particles[species][species_data])

            if inputIteration.particles[species][species_data].scalar:
                
                component_data = inputIteration.particles[species][species_data][io.Record_Component.SCALAR][:]
                readSeries.flush()
                output_component_data = outputIteration.particles[species][species_data]
                
                if 'value' in inputIteration.particles[species][species_data].attributes:
                    value = inputIteration.particles[species][species_data].get_attribute('value')
                    output_component_data[io.Record_Component.SCALAR].make_constant(value)
                
                else:
                    dataset = io.Dataset( component_data.dtype, component_data.shape )
                    output_component_data[io.Record_Component.SCALAR].reset_dataset(dataset)
                    output_component_data[io.Record_Component.SCALAR][()] = component_data

            else:
                output_species_component = outputIteration.particles[species][species_data]
                for component in inputIteration.particles[species][species_data]:
                    #component is 'x', 'y' or 'z'
                    
                    output_component_data = output_species_component[component]
                    copy_attributes(inputIteration.particles[species][species_data][component], output_component_data)
                    
                    component_data = inputIteration.particles[species][species_data][component][:]
                    readSeries.flush()
                    
                    dataset = io.Dataset( component_data.dtype, component_data.shape )
                    output_component_data.reset_dataset(dataset)
                    
                    output_component_data.store_chunk(component_data)
            writeSeries.flush()

            
        for patch_component in inputIteration.particles[species].particle_patches:
            #print(patch_component)
            if inputIteration.particles[species].particle_patches[patch_component].scalar:
                output_component_data = outputIteration.particles[species].particle_patches[patch_component]
                # read-only !
                copy_attributes(inputIteration.particles[species].particle_patches[patch_component], output_component_data)

                component_data = inputIteration.particles[species].particle_patches[patch_component][io.Record_Component.SCALAR].load()
                readSeries.flush()
                #print(component_data, component_data.dtype, component_data.shape)

                dataset = io.Dataset( component_data.dtype, component_data.shape )
                output_component_data[io.Record_Component.SCALAR].reset_dataset(dataset)
                #outputIteration.particles[species].particle_patches[patch_component][io.Record_Component.SCALAR].reset_dataset(dataset)

                for i in range(len(component_data)):
                    output_component_data[io.Record_Component.SCALAR].store(i, component_data[i])
                    #outputIteration.particles[species].particle_patches[patch_component][io.Record_Component.SCALAR].store(i, component_data[i])

            else:
                copy_attributes(inputIteration.particles[species].particle_patches[patch_component], outputIteration.particles[species].particle_patches[patch_component])

                output_species_component = outputIteration.particles[species].particle_patches[patch_component]
                for component in inputIteration.particles[species].particle_patches[patch_component]:
                    #component is 'x', 'y' or 'z'
                    #print(component)
                    output_component_data = output_species_component[component]
                    copy_attributes(inputIteration.particles[species].particle_patches[patch_component][component], output_component_data)

                    component_data = inputIteration.particles[species].particle_patches[patch_component][component].load()
                    readSeries.flush()

                    dataset = io.Dataset( component_data.dtype, component_data.shape )
                    output_component_data.reset_dataset(dataset)

                    for i in range(len(component_data)):
                        output_component_data.store(i, component_data[i])
        
        # can't write particles patches before all data is set
        writeSeries.flush()
        
        
def copy_field_data(inputIteration, outputIteration):
    copy_attributes(inputIteration.meshes, outputIteration.meshes)

    for field_name in iteration.meshes:

        if field_name in ['E', 'B']:
            #print(field_name)
            copy_attributes(inputIteration.meshes[field_name], outputIteration.meshes[field_name])
            
            for component in inputIteration.meshes[field_name]:
                #print("     ", component)
                copy_attributes(inputIteration.meshes[field_name][component], outputIteration.meshes[field_name][component])
                
                dataset = io.Dataset( ex.dtype, ex.reshape(zdim, ydim, xdim).shape )    # or ey, ez, bx, by or bz instead
                
                if field_name == 'E':
                    if component == 'x':
                        data = bx
                    elif component == 'y':
                        data = by
                    elif component == 'z':
                        data = bz
                        
                elif field_name == 'B':
                    if component == 'x':
                        data = bx
                    elif component == 'y':
                        data = by
                    elif component == 'z':
                        data = bz
                
                outputIteration.meshes[field_name][component].reset_dataset(dataset)
                outputIteration.meshes[field_name][component].store_chunk(data.reshape(zdim, ydim, xdim))
                writeSeries.flush()
            continue

        #print(field_name)
        field = outputIteration.meshes[field_name]
        copy_attributes(iteration.meshes[field_name], field)

        if field_name == 'picongpu_idProvider':
            for component in inputIteration.meshes[field_name]:

                copy_attributes(inputIteration.meshes[field_name][component], field[component])

                data = inputIteration.meshes[field_name][component].load_chunk()

                dataset = io.Dataset(data.dtype, data.shape)
                field[component].reset_dataset(dataset)

                field[component].store_chunk(data)
            continue

        if inputIteration.meshes[field_name].scalar:
            data = inputIteration.meshes[field_name][io.Record_Component.SCALAR].load_chunk()
            readSeries.flush()

            dataset = io.Dataset( data.dtype, data.shape)
            field[io.Record_Component.SCALAR].reset_dataset(dataset)

            field[io.Record_Component.SCALAR].store_chunk()

        else:

            for component in inputIteration.meshes[field_name]:
                #print("   ",component)

                data = inputIteration.meshes[field_name][component].load_chunk()
                readSeries.flush()
                dataset = io.Dataset(data.dtype, data.shape)

                output_data = outputIteration.meshes[field_name][component]
                output_data.reset_dataset(dataset)
                output_data.store_chunk(data)

        writeSeries.flush()

In [ ]:
def write():
    
    global writeSeries
    
    dim = ['x', 'y', 'z']

    # copy general attributes ----------------------------------
    copy_series_attributes(readSeries, writeSeries)

    outputIteration = writeSeries.iterations[0]

    copy_attributes(iteration, outputIteration)
    
    # copy particle data
    copy_particle_data(iteration, outputIteration)
    
    # copy/write field data
    copy_field_data(iteration, outputIteration)
    
    del writeSeries

In [ ]:
def smoothFields(ex, ey, ez, bx, by, bz, sigma=4):
    ex = gaussian_filter(ex.reshape(zdim, ydim, xdim), sigma)
    ey = gaussian_filter(ey.reshape(zdim, ydim, xdim), sigma)
    ez = gaussian_filter(ez.reshape(zdim, ydim, xdim), sigma)
    bx = gaussian_filter(bx.reshape(zdim, ydim, xdim), sigma)
    by = gaussian_filter(by.reshape(zdim, ydim, xdim), sigma)
    bz = gaussian_filter(bz.reshape(zdim, ydim, xdim), sigma)

## load particle data from the checkpoint
define the species identifier string, as in the simulation

In [ ]:
species = "b"

xpos_incell = iteration.particles[species]["position"]["x"][:]
ypos_incell = iteration.particles[species]["position"]["y"][:]
zpos_incell = iteration.particles[species]["position"]["z"][:]
xpos_offset = iteration.particles[species]["positionOffset"]["x"][:]
ypos_offset = iteration.particles[species]["positionOffset"]["y"][:]
zpos_offset = iteration.particles[species]["positionOffset"]["z"][:]
momentumx = iteration.particles[species]["momentum"]["x"][:]
momentumy = iteration.particles[species]["momentum"]["y"][:]
momentumz = iteration.particles[species]["momentum"]["z"][:]
weightings = iteration.particles[species]["weighting"][io.Record_Component.SCALAR][:]
charge = iteration.particles[species]["charge"][io.Record_Component.SCALAR][:]
mass = iteration.particles[species]["mass"][io.Record_Component.SCALAR][:]

readSeries.flush()

xpos = xpos_incell + np.float32(xpos_offset)
ypos = ypos_incell + np.float32(ypos_offset)
zpos = zpos_incell + np.float32(zpos_offset)

# free some memory
del xpos_incell, ypos_incell, zpos_incell
del xpos_offset, ypos_offset, zpos_offset


# use just every nth particle
# needs to be in a new numpy array
# because it must be continous in memory to be copied to gpu
momentumx = np.array(momentumx[::nth])
momentumy = np.array(momentumy[::nth])
momentumz = np.array(momentumz[::nth])
weightings = np.array(weightings[::nth])
charge = np.array(charge[::nth])
mass = np.array(mass[::nth])
xpos = np.array(xpos[::nth])
ypos = np.array(ypos[::nth])
zpos = np.array(zpos[::nth])
particleCount = len(mass)

## output to check particleCount and gridsize

In [ ]:
print("particleCount:", particleCount)
fieldShape = iteration.meshes["E"]["x"][:].shape
xdim = fieldShape[2]
ydim = fieldShape[1]
zdim = fieldShape[0]
shape = xdim * ydim * zdim
print("gridSize (z,y,x, total):", fieldShape, shape)

## calculate the fields
maybe set the the parallelization mehtod (over fields or particles) manually

In [ ]:

if particleCount <= shape:    # maybe add a constant, but needs testing for optimal choise
    # calculate every field position in parallel (loop over the particles)
    # few particles, many field positions
    blockdim = 256                                              # number of threads per block (multiple 32 for optimal speed)
    griddim = math.ceil(shape / blockdim)                       # number of blocks in the grid
    cudaFunc = FieldParallel
    print("Fields in parallel")
    
else:
    # calculate every particle in parallel (loop over the entire field)
    # many particles, few field positions
    blockdim = 32  #256                                         # number of threads per block (multiple 32 for optimal speed)
    griddim = math.ceil(particleCount / blockdim)         # number of blocks in the grid
    cudaFunc = particleParallel
    print("particles in parallel")
    
print("particles to be processed:", particleCount)

# for test on a smaller number of particles
#blockdim = 1
#griddim = 1

array_dtype = np.float32                            # dtype for arrays
ex = np.zeros(shape, dtype=array_dtype)
ey = np.zeros(shape, dtype=array_dtype)
ez = np.zeros(shape, dtype=array_dtype)
bx = np.zeros(shape, dtype=array_dtype)
by = np.zeros(shape, dtype=array_dtype)
bz = np.zeros(shape, dtype=array_dtype)

# All variables with _d at the and are for device variables
print("allocate/copy field/data arrays to device")
ex_d = cuda.to_device(ex)
ey_d = cuda.to_device(ey)
ez_d = cuda.to_device(ez)
bx_d = cuda.to_device(bx)
by_d = cuda.to_device(by)
bz_d = cuda.to_device(bz)

charge_d = cuda.to_device(charge)
xpos_d = cuda.to_device(xpos)
ypos_d = cuda.to_device(ypos)
zpos_d = cuda.to_device(zpos)
momentumx_d = cuda.to_device(momentumx)
momentumy_d = cuda.to_device(momentumy)
momentumz_d = cuda.to_device(momentumz)
mass_d = cuda.to_device(mass)
weightings_d = cuda.to_device(weightings)

print("\nstart of field calculation time:", time.ctime())
starttime = time.time()

#cudaFunc[griddim, blockdim](ex, ey, ez, bx, by, bz, charge, xpos, ypos, zpos, momentumx, momentumy, momentumz, mass, weightings, xdim, ydim, zdim, particleCount)
cudaFunc[griddim, blockdim](ex_d, ey_d, ez_d, bx_d, by_d, bz_d, charge_d, xpos_d, ypos_d, zpos_d, momentumx_d, momentumy_d, momentumz_d, mass_d, weightings_d, xdim, ydim, zdim, particleCount)

exeTime = time.time()-starttime
print("time: {:.5}".format( exeTime ), "s")
print("avgTime per particle per cell:", exeTime / particleCount / (xdim*ydim*zdim), "s/(particle*cell)")

## if the kernel dies here, start the jupyter session with more memory
#floatSize = 4 # 4 for float32, 8 for float64
#print("min.", shape * 6 * floatSize , "GB for field arrays")

print("\nget field arrays from device", time.ctime())
ex_d.copy_to_host(ex)
ey_d.copy_to_host(ey)
ez_d.copy_to_host(ez)
bx_d.copy_to_host(bx)
by_d.copy_to_host(by)
bz_d.copy_to_host(bz)
print("field arrays copied from device to host", time.ctime())

# delete device variables, no longer needed after the parallel computation
del charge_d
del xpos_d
del ypos_d
del zpos_d
del momentumx_d
del momentumy_d
del momentumz_d
del mass_d
del weightings_d

if write == True:
    smoothFields(ex, ey, ez, bx, by, bz)
    
    write()
    
    print("\nwrote to checkpoint")
else:
    print("\nno data was written to the checkpoint")

## Write fields to the checkpoint manually

In [ ]:
# only smooth if the fields aren't already smoothed in the automatic writing
if do_write == False:
    print("smoothing fields... may take some time")
    smoothFields(ex, ey, ez, bx, by, bz)
    print("Fields were smoothed")
else:
    print("nothing happend, already smoothed")

In [ ]:
# manual write
if do_write == False:    
    write()
    print("data written")
    
else:
    print("nothing happened, data was already written")

# Visualization of the fields if necessary

and other stuff, not realy documented with comments

## absolute field values

In [ ]:
# absolute values of the fields
ef = np.sqrt(ex**2+ey**2+ez**2)
bf = np.sqrt(bx**2+by**2+bz**2)

In [ ]:
figsize(10, 10)
depth = 400
imshow((ef.reshape(zdim, ydim, xdim)[depth].T))
colorbar()
show()

## show particle positions

In [ ]:
figsize(15,15)
a = np.histogram2d(xpos, ypos, weights=weightings, bins=[np.linspace(0, xdim, xdim+1), np.linspace(0, ydim, ydim+1)] )
imshow(a[0])
xlabel("y cells")
ylabel("x cells")
show()

In [ ]:
del ex_d
del ey_d
del ez_d
del bx_d
del by_d
del bz_d